Домашнее задание 3.

In [52]:
import numpy as np
import pandas as pd
import matplotlib as plt
from collections import Counter

Задание 3.

In [78]:
def entropy(classes):
    unique_classes = np.unique(classes)
    probabilities = np.array([0.1] * len(unique_classes))
    i = 0
    for cl in unique_classes:
        n = len(classes[classes == cl])
        probabilities[i] = (n + 0.1 - 0.1) / len(classes)
        i += 1
    return -np.sum(probabilities * np.log2(probabilities))


def gini(classes):
    unique_classes = np.unique(classes)
    probabilities = np.array([0.1] * len(unique_classes))
    i = 0
    for cl in unique_classes:
        n = len(classes[classes == cl])
        probabilities[i] = (n + 0.1 - 0.1) / len(classes)
        i += 1
    return np.sum(probabilities * (1 - probabilities))


def miss_error(classes):
    max = Counter(y).most_common(1)[0][1] / len(classes)
    return 1 - max


impurities = {"entropy": entropy, "gini": gini, "miss_error": miss_error}

features = []

class Tree:
    children = []
    isLeaf = True

    def __init__(self, data, impurity, max_depth, depth = 0):
        features = []
        y = data[:, 0]
        x = data[:, 1:]
        if len(np.unique(y)) == 1:
            self.isLeaf = True
            self.cl = np.unique(y)[0]
        elif depth == max_depth:
            self.isLeaf = True
            cl = Counter(y).most_common(1)[0][0]
            self.cl = cl
        else:
            maxIG = 0
            I_node = impurity(y)
            for i in range(len(x[0])):
                if i not in features:
                    feature_data = x[:, i]
                    for j in range(len(feature_data)):
                        left = y[feature_data <= feature_data[j]]
                        right = y[feature_data > feature_data[j]]
                        if len(left) == 0 or len(right) == 0:
                            continue
                        I_left = impurity(left) * len(left) / len(y)
                        I_right = impurity(right) * len(right) / len(y)
                        IG = I_node - I_left - I_right
                        if maxIG < IG:
                            maxIG = IG
                            self.sep_feature = i
                            self.separator = feature_data[j]
            if maxIG > 0:
                features.append(self.sep_feature)
                self.isLeaf = False
                left = np.array([False] * len(data))
                right = np.array([False] * len(data))
                for i in range(len(y)):
                    if data[i][self.sep_feature + 1] <= self.separator:
                        left[i] = True
                    else:
                        right[i] = True
                self.children.append(Tree(data[left], impurity, max_depth, depth + 1))
                self.children.append(Tree(data[right], impurity, max_depth, depth + 1))
            else:
                self.isLeaf = True
                cl = Counter(y).most_common(1)[0][0]
                self.cl = cl

    def classify(self, x):
        if self.isLeaf:
            return self.cl
        else:
            if x[self.sep_feature] <= self.separator:
                return self.children[0].classify(x)
            else:
                return self.children[1].classify(x)

In [54]:
data = np.array(pd.read_csv('/Users/user/PycharmProjects/MLHW1/cancer.csv'))


In [55]:
print ("Энтропия")
for j in range(1, 11):
    np.random.shuffle(data)
    data_tr = data[:400]
    data_eval = data[400:]
    print("depth: " + str(j), end = ' Accuracy: ')
    desission_tree = Tree(data_tr, impurities['entropy'], j)
    y = data_eval[:, 0]
    y_received = []
    x = data_eval[:, 1:]
    for i in range(len(data_eval)):
        y_received.append(desission_tree.classify(x[i]))
    y_received = np.array(y_received)
    k = 0
    for i in range(len(y_received)):
        if y_received[i] == y[i]:
            k += 1
    print (k / len(y_received))

Энтропия
depth: 1 Accuracy: 

0.9053254437869822
depth: 2 Accuracy: 

0.9053254437869822
depth: 3 Accuracy: 

0.9112426035502958
depth: 4 Accuracy: 

0.9053254437869822
depth: 5 Accuracy: 

...

0.893491124260355
depth: 7 Accuracy: 

0.8816568047337278
depth: 8 Accuracy: 

0.863905325443787
depth: 9 Accuracy: 

0.893491124260355
depth: 10 Accuracy: 

0.8994082840236687


In [56]:
print ("Джини")
for j in range(1, 11):
    np.random.shuffle(data)
    data_tr = data[:400]
    data_eval = data[400:]
    print("depth: " + str(j), end = ' Accuracy: ')
    decision_tree = Tree(data_tr, impurities['gini'], j)
    y = data_eval[:, 0]
    y_received = []
    x = data_eval[:, 1:]
    for i in range(len(data_eval)):
        y_received.append(decision_tree.classify(x[i]))
    y_received = np.array(y_received)
    k = 0
    for i in range(len(y_received)):
        if y_received[i] == y[i]:
            k += 1
    print (k / len(y_received))
    node = None

Джини
depth: 1 Accuracy: 

0.9112426035502958
depth: 2 Accuracy: 

0.8816568047337278
depth: 3 Accuracy: 

0.9053254437869822
depth: 4 Accuracy: 

0.9289940828402367
depth: 5 Accuracy: 

...

0.8875739644970414
depth: 7 Accuracy: 

0.8994082840236687
depth: 8 Accuracy: 

0.9171597633136095
depth: 9 Accuracy: 

0.8757396449704142
depth: 10 Accuracy: 

0.8875739644970414


In [66]:
print ("Misclassification error")
for j in range(1, 11):
    np.random.shuffle(data)
    data_tr = data[:400]
    data_eval = data[400:]
    print("depth: " + str(j), end = ' Accuracy: ')
    desission_tree = Tree(data_tr, impurities['miss_error'], j)
    y = data_eval[:, 0]
    y_received = []
    x = data_eval[:, 1:]
    for i in range(len(data_eval)):
        y_received.append(desission_tree.classify(x[i]))
    y_received = np.array(y_received)
    k = 0
    for i in range(len(y_received)):
        if y_received[i] == y[i]:
            k += 1
    print (k / len(y_received))
    node = None

Misclassification error
depth: 1 Accuracy: 

0.9053254437869822
depth: 2 Accuracy: 

0.9112426035502958
depth: 3 Accuracy: 

0.893491124260355
depth: 4 Accuracy: 

0.8816568047337278
depth: 5 Accuracy: 

...

0.9349112426035503
depth: 7 Accuracy: 

0.893491124260355
depth: 8 Accuracy: 

0.893491124260355
depth: 9 Accuracy: 

0.9112426035502958
depth: 10 Accuracy: 

0.9171597633136095


In [67]:
data = np.array(pd.read_csv('/Users/user/PycharmProjects/MLHW1/spam.csv'))
y = data[:, len(data[0]) - 1]
x =  data[:, :len(data[0]) - 1]
data = np.column_stack((y, x))

In [76]:
print ("Энтропия")
for j in range(1, 11):
    np.random.shuffle(data)
    data_tr = data[400:]
    data_eval = data[:400]
    print("depth: " + str(j), end = ' Accuracy: ')
    desission_tree = Tree(data_tr, impurities['entropy'], j)
    y = data_eval[:, 0]
    y_received = []
    x = data_eval[:, 1:]
    for i in range(len(data_eval)):
        y_received.append(desission_tree.classify(x[i]))
    y_received = np.array(y_received)
    k = 0
    for i in range(len(y_received)):
        if y_received[i] == y[i]:
            k += 1
    print (k / len(y_received))

Энтропия
depth: 1 Accuracy: 

0.7825
depth: 2 Accuracy: 

0.785
depth: 3 Accuracy: 

0.77
depth: 4 Accuracy: 

0.78
depth: 5 Accuracy: 

...

0.765
depth: 7 Accuracy: 

0.78
depth: 8 Accuracy: 

0.7925
depth: 9 Accuracy: 

0.7575
depth: 10 Accuracy: 

0.7875


In [79]:
print ("Джини")
for j in range(1, 11):
    np.random.shuffle(data)
    data_tr = data[400:]
    data_eval = data[:400]
    print("depth: " + str(j), end = ' Accuracy: ')
    decision_tree = Tree(data_tr, impurities['gini'], j)
    y = data_eval[:, 0]
    y_received = []
    x = data_eval[:, 1:]
    for i in range(len(data_eval)):
        y_received.append(decision_tree.classify(x[i]))
    y_received = np.array(y_received)
    k = 0
    for i in range(len(y_received)):
        if y_received[i] == y[i]:
            k += 1
    print (k / len(y_received))


Джини
depth: 1 Accuracy: 

0.7875
depth: 2 Accuracy: 

0.8225
depth: 3 Accuracy: 

0.8
depth: 4 Accuracy: 

0.7725
depth: 5 Accuracy: 

0.805
depth: 6 Accuracy: 

0.76
depth: 7 Accuracy: 

0.7975
depth: 8 Accuracy: 

0.7725
depth: 9 Accuracy: 

0.75
depth: 10 Accuracy: 

In [ ]:
print ("Misclassification error")
for j in range(1, 11):
    np.random.shuffle(data)
    data_tr = data[:400]
    data_eval = data[400:]
    print("depth: " + str(j), end = ' Accuracy: ')
    desission_tree = Tree(data_tr, impurities['miss_error'], j)
    y = data_eval[:, 0]
    y_received = []
    x = data_eval[:, 1:]
    for i in range(len(data_eval)):
        y_received.append(desission_tree.classify(x[i]))
    y_received = np.array(y_received)
    k = 0
    for i in range(len(y_received)):
        if y_received[i] == y[i]:
            k += 1
    print (k / len(y_received))
    node = None